# Bangumi Takeout

项目主页：[jerrylususu/bangmui-takeout-py](https://github.com/jerrylususu/bangumi-takeout-py)

## 运行步骤

1. 请点击菜单栏「代码执行程序 - 全部运行」（繁體中文「執行階段－全部執行」）
2. 卷动到页面底部，点击认证链接并选择「创建个人令牌 - 任意名字/7天 - Submit」，将下一个页面中的 `access_token` 复制到页面底部的表单中 Token 填写处
3. 选择数据源，具体请参见项目主页，如无特殊要求保持默认即可
4. 选择时区偏移。在 Colab 上运行时，若不设置此项会导致输出结果中时区为 UTC。如无特殊需要可保持默认 `+8` 不变
5. 如果有之前导出的 `takeout.json` 文件，可点击「Upload」上传，此时会采用增量方式更新收视进度，能极大提升导出速度。如无可忽略
6. 选择导出格式：`json`, `html`，`csv`
7. 最后点击「Run」以启动导出过程

## 常见问题

表单可能需要稍等一会才会显示（需要等待运行时分配，进度会在右上角显示），请耐心等待，最长应该不会超过一分钟。

如无特殊需求，数据源建议保持默认的「Bangumi Archive」以减小对服务器的压力。关于两个数据源之间的差异，请参见本项目主页的「数据源」一节。

如果选择上传此前的 `takeout.json`，上传过程中没有进度显示，需要稍等一会。也可以用 Colab 界面左侧的文件面板上传。

执行完成后会自动触发 `takeout.json`, `takeout.html` 或 `takeout.csv` 的下载，也可以从左侧文件侧边栏（最后一个图标）中手动下载。如果浏览器提示「下载多个文件」，请选择允许。

In [ ]:
from ipywidgets import interact, widgets
from IPython.display import display
import json
from google.colab import files

display(widgets.HTML(value='基本参数'))


token_html = widgets.HTML(value='<a href="https://api.bgm.tv/v0/oauth/redirect" target="_blank">点击此链接生成 Access Token</a>')
display(token_html)

text = widgets.Text(value='',placeholder='请将 Access Token 复制到此',description='Token:',disabled=False)
display(text)

select = widgets.Dropdown(options=['Bangumi API', 'Bangumi Archive'],value='Bangumi Archive',description='数据源:',disabled=False,)
display(select)

timezone_offset_text = widgets.Text(value='+8',placeholder='输出使用的时区，从 -12 到 12',description='时区偏移:',disabled=False)
display(timezone_offset_text)

display(widgets.HTML(value='如有，请上传之前导出的 takeout.json。'))


existing_upload = widgets.FileUpload(accept='.json',multiple=False)
display(existing_upload)

display(widgets.HTML(value='输出格式'))


checkbox_as_json = widgets.Checkbox(value=True, description='JSON', indent=True)
display(checkbox_as_json)
checkbox_as_html = widgets.Checkbox(value=True, description='HTML', indent=True)
display(checkbox_as_html)
checkbox_as_csv = widgets.Checkbox(value=True, description='CSV', indent=True)
display(checkbox_as_csv)


button = widgets.Button(tooltip='运行',description="运行")
display(button)

def callback(wdgt):
    print("uploading... may take a while...")
    with open("takeout.json", "wb") as fp:
      for k, v in existing_upload.value.items():
        fp.write(v["content"])
        break
    print("upload finished!")

    !git clone https://github.com/jerrylususu/bangumi-takeout-py
    !pip install requests tqdm
    !mv bangumi-takeout-py/* .
    !rm -rf bangumi-takeout-py
    !touch no_gui

    import fetch
    import generate_html
    import generate_csv
    from utils import get_newest_archive

    if select.value == "Bangumi Archive":
      print("downloading archive data")
      download_url = get_newest_archive()
      wget_result = !wget {download_url} -O dump.zip
      unzip_result = !unzip dump.zip

    with open(".bgm_token","w",encoding="u8") as f:
      tokens = {"access_token": text.value}
      json.dump(tokens, f)
    
    fetch.main()

    tz_offset_hours = int(timezone_offset_text.value)

    if checkbox_as_json.value:
      files.download('takeout.json')

    if checkbox_as_html.value:
      generate_html.main(offset_hours=tz_offset_hours)
      files.download('takeout.html') 

    if checkbox_as_csv.value:
      generate_csv.main(offset_hours=tz_offset_hours)
      files.download('takeout.csv')

button.on_click(callback)